## Create Sampled Dataset of PC Urban


In [5]:
import argparse
import os
import yaml
import numpy as np
import numpy.linalg as la
import open3d as o3d
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from plyfile import PlyData, PlyElement
from matplotlib import cm
from functools import partial
import concurrent.futures
import random

from sphere import Sphere
from dh_grid import DHGrid
from laserscan import SemLaserScan

import os
import time
import matplotlib.pyplot as plt
import numpy as np
import json
import pprint
from os.path import join
import glob
import cv2
import open3d as o3d


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dataroot = 'pc urban/'
segments = os.listdir(dataroot)

print(f'Found {len(segments)} elements in the folder.')

Found 2 elements in the folder.


In [31]:
def retrieve_clouds(lidar_file, label_file):
        # print(f'Loading from {lidar_file} and {label_file}')
        plydata = PlyData.read(lidar_file)
        vertex = plydata['vertex']
        x = vertex['x']
        y = vertex['y']
        z = vertex['z']        
        i = np.zeros_like(x)
        classes_and_instance = np.genfromtxt(label_file)
        # print(f'classes shape is {classes_and_instance.shape}')
        c = classes_and_instance[:,0]                
        return np.column_stack((x,y,z,i,c))

def extract_label_file(lidar_file):
    file_name_semantic_label = lidar_file.split('/')
    file_name_semantic_label = file_name_semantic_label[-1].split('.')        
    file_name_semantic_label = file_name_semantic_label[0] + '.' + file_name_semantic_label[1] + 'Label.txt'                      
    return file_name_semantic_label

all_sem_clouds = []
for segment in tqdm(segments):
    cloud_path = f'{dataroot}/{segment}/PLY/*.ply'
    lidar_files = sorted(glob.glob(cloud_path))
        
    for lidar_file in lidar_files:
        label_file = extract_label_file(lidar_file)
        label_file = f'{dataroot}/{segment}/Label/{label_file}'
        cloud = retrieve_clouds(lidar_file, label_file)
        all_sem_clouds.append(cloud)    
print(f'Extracted {len(all_sem_clouds)} clouds')

  0%|          | 0/2 [00:00<?, ?it/s]

Extracted 573 clouds


In [37]:
def progresser(sample_idx, grid, auto_position=True, write_safe=False, blocking=True, progress=False):
    sample = all_sem_clouds[sample_idx]
    sample_sphere = Sphere(sample)
    features = sample_sphere.sampleUsingGrid(grid)
    return features

bw = 50
grid, _ = DHGrid.CreateGrid(bw)
print(f"Loading complete. Computing features...")
# parallel
sem_idx = np.arange(0, len(all_sem_clouds))
sample_func = partial(progresser, grid=grid)
sem_features = process_map(sample_func, sem_idx, max_workers=16)            

filename = f"{dataroot}/clouds.npy"
np.save(filename, sem_features)
print(f"Wrote features to {filename}.")

filename = f'archive'
np.save(f'{dataroot}/{filename}.npy', all_sem_clouds)
print(f'Saved clouds to {dataroot}/{filename}.npy')

Loading complete. Computing features...


  0%|          | 0/573 [00:00<?, ?it/s]

Wrote features to /media/berlukas/Data/data/datasets/s2ae/pc urban//clouds.npy.


/usr/lib/python3/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Saved clouds to /media/berlukas/Data/data/datasets/s2ae/pc urban//archive.npy


## Visualize extracted pointclouds

In [42]:
def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

def visualizeRawPointcloud(pcl, val):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl[:, 0:3])
    colors = mapIntensityToRGB(val)
#     colors = scan.sem_color_lut[pcl[:,4].astype(np.int)]
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.visualization.draw_geometries([pcd], width=640,  height=480)    
    
def createGrid_old(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([n_grid * n_grid, 2])
    for i in range(n_grid):
        for j in range(n_grid):
            points[k, 0] = (np.pi*(2*i+1))/(4*bw)
            points[k, 1] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])
    return cart_grid

def create_sampling_sphere(bw):
    grid = createGrid_old(bw)
    xyz_grid = convertGridToEuclidean_old(grid)
    intensities = np.zeros((xyz_grid.shape[0],1))
    sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
    return sampling_grid.T

In [40]:
pc = all_sem_clouds[11]
visualizeRawPointcloud(pc, pc[:, 4])

In [36]:
classes = np.unique(pc[:, 4])
print(f'we have these classes: {classes}')

we have these classes: [ 0.  1.  8.  9. 11. 15. 28. 30.]


In [43]:
cur_sem_cloud = sem_features[11]

cur_sem_cloud = np.reshape(cur_sem_cloud, (3, -1)).T
pc = create_sampling_sphere(bw)
points_xyz = pc.T[:,0:3]
points_xyzl = np.column_stack((points_xyz, cur_sem_cloud[:,2]))

visualizeRawPointcloud(points_xyzl, points_xyzl[:, 3])